# First Agent

In [1]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

In [2]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
load_dotenv(override=True)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = init_chat_model("openai:gpt-4o-mini")

In [ ]:
def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()


In [8]:
def chat(user_input):
    initial_state = {"messages": [{'role': 'user', 'content': user_input}]}
    result = graph.invoke(initial_state)
    return result['messages'][-1].content

chat("Hello, how are you?")

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"